In [36]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
# assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 11

import numpy as np
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          TrainingArguments, Trainer, DataCollatorWithPadding, TrainingArguments)
from datasets import Dataset, load_dataset, load_metric
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

dataset = load_dataset('csv', data_files=r'train\track_a\eng.csv')
dataset2 = load_dataset('knowledgator/events_classification_biotech')
print(dataset2)

classes2 = [class_ for class_ in dataset2['train'].features['label 1'].names if class_]
print(classes2)

# class_weights = torch.tensor([1.8, 0.7, 1.2, 0.9, 0.9])

# loss_fn = CrossEntropyLoss(weight=class_weights)

classes = ["Anger", "Fear", "Joy", "Sadness", "Surprise"]
class2id2 = {class_:id for id, class_ in enumerate(classes2)}
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

c:\Users\thato\miniconda3\envs\semeval1\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for knowledgator/events_classification_biotech contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/knowledgator/events_classification_biotech
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5'],
        num_rows: 2759
    })
    test: Dataset({
        features: ['title', 'content', 'target organization', 'all_labels', 'all_labels_concat', 'label 1', 'label 2', 'label 3', 'label 4', 'label 5'],
        num_rows: 381
    })
})
['event organization', 'executive statement', 'regulatory approval', 'hiring', 'foundation', 'closing', 'partnerships & alliances', 'expanding industry', 'new initiatives or programs', 'm&a', 'service & product providing', 'new initiatives & programs', 'subsidiary establishment', 'product launching & presentation', 'product updates', 'executive appointment', 'alliance & partnership', 'ipo exit', 'article publication', 'clinical trial sponsorship', 'company description', 'investment in public company', 'other', 'expanding geography', 'participation in an event', 'support 

In [37]:
from transformers import AutoTokenizer
 
model_path = 'microsoft/deberta-v3-small'

tokenizer = AutoTokenizer.from_pretrained(model_path, clean_up_tokenization_spaces=True)

c:\Users\thato\miniconda3\envs\semeval1\Lib\site-packages\transformers\convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [38]:
# def preprocess_function(example):
#    text = example['text']
#    labels = [0. for i in range(len(classes))]
#    for label in classes:
#        id = class2id[label]
#        labels[id] = float(example[label])
  
#    example = tokenizer(text, truncation=True)
#    example['labels'] = labels
#    return example

examples = []
y_true = []

for example in dataset['train']:
  examples.append(example['text'])
  y_true.append([float(example['Anger']), float(example['Fear']), float(example['Joy']), float(example['Sadness']), float(example['Surprise'])])

# make training and validation sets
examples_train, examples_test, labels_train, labels_test = train_test_split(examples, y_true, test_size=0.05, random_state=42)

print(examples)
print(examples_train)
print(examples_test)

train_dataset = Dataset.from_dict( {"text": examples_train, "label": labels_train} )
test_dataset = Dataset.from_dict( {"text": examples_test, "label": labels_test} )

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# make Datasets
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# tokenized_test = test_dataset.map(preprocess_function)

['But not very happy.', "Well she's not gon na last the whole song like that, so since I'm behind her and the audience can't see below my torso pretty much, I use my hand to push down on the lid and support her weight.", "She sat at her Papa's recliner sofa only to move next to me and start clinging to my arms.", 'Yes, the Oklahoma city bombing.', 'They were dancing to Bolero.', 'Still had sex with her, though.', "But I am exhausted-my eyes feel like they are about to pop out of my head-I need some soothing music and images to help my mediate, even for a few minutes, just take me away $ # 365 yoga Monday's Musical Meditation: 0 minutes ago early night-first day of work after mat.", 'We ordered some food at Mcdonalds instead of buying food at the theatre because of the ridiculous prices the theatre has.', 'Now my parents live in the foothills, and the college is in a large valley.', 'We get to the porch and my dog starts *growling*, like a big boy growl, like shits going down growl.', '

Map:   0%|          | 0/2629 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/139 [00:00<?, ? examples/s]

In [39]:
import evaluate
import numpy as np

from transformers import DataCollatorWithPadding

# print(tokenized_dataset)
# tokenized_dataset = tokenized_dataset["train"].train_test_split(test_size=0.6) 
# print(tokenized_dataset)
# training!
model = AutoModelForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification", num_labels=5)

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

mini_tokenized_train = tokenized_train.select(range(100))
mini_tokenized_test = tokenized_test.select(range(5))

trainer = Trainer(model=model, args=training_args, train_dataset=mini_tokenized_train, eval_dataset=mini_tokenized_test)
# trainer = CustomTrainer(model=model, args=training_args, train_dataset=mini_tokenized_train, eval_dataset=mini_tokenized_test)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):

   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))
#references=labels.astype(int).reshape(-1))

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(

   model_path, num_labels=len(classes),
           id2label=id2class, label2id=class2id,
                       problem_type = "multi_label_classification")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
possible_learn_rate = [2e-4, 2e-5, 2e-6]
possible_batch_size = [8, 16, 32]


for learn_r in possible_learn_rate:
   for batch_s in possible_batch_size:
      training_args = TrainingArguments(
         output_dir="my_awesome_model",
         per_device_train_batch_size= batch_s
         learning_rate= learn_r,
         # per_device_train_batch_size=3,
         # per_device_eval_batch_size=3,
         num_train_epochs=3,
         weight_decay=0.01,
         evaluation_strategy="epoch",
         save_strategy="epoch",
         load_best_model_at_end=True,
      )

      trainer = Trainer(

         model=model,
         args=training_args,
         train_dataset=tokenized_train,
         eval_dataset=tokenized_test      ,
         tokenizer=tokenizer,
         data_collator=data_collator,
         compute_metrics=compute_metrics,
         # mini_batch_sizee = b_s
      )

      trainer.train()
      predictions = trainer.predict(tokenized_test)
      probs = sigmoid(torch.from_numpy(predictions.predictions))
      # print("PROBS: ", probs)
      # print("LABELS: ", torch.tensor(tokenized_test['label'])) # trues
      print(f"LOG: Learn: {learn_r} Batch: {batch_s}")
      for thresh in [0.35, 0.4, 0.45, 0.5]:
      # binarize predictions
         binary_predictions = (probs >= thresh).long()
         print("THRESH = ", thresh)
         # print("PREDS: ", binary_predictions)

         print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='weighted'))
         print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='micro'))
         print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='macro'))
         print("==============")
      nums = np.round(np.linspace(.3, .7, 40), 2)
      final_nums = np.append(nums, 0.5)

      best_thresh = 0
      best_f1 = 0
      for curr_thresh in final_nums:
      # binarize predictions
         binary_predictions = (probs >= curr_thresh).long()
         
         # print("PREDS: ", binary_predictions)

         curr_f1 = f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='weighted')
         if curr_f1 > best_f1:
            best_f1 = curr_f1
            best_thresh = curr_thresh
      # print("THRESH = ", curr_thresh, " F1 = ", curr_f1)
      print(f"F1: {best_f1}")
      print(best_thresh)

c:\Users\thato\miniconda3\envs\semeval1\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/987 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.4443199038505554, 'eval_accuracy': 0.7870503597122303, 'eval_f1': 0.645933014354067, 'eval_precision': 0.6585365853658537, 'eval_recall': 0.6338028169014085, 'eval_runtime': 3.0257, 'eval_samples_per_second': 45.94, 'eval_steps_per_second': 5.949, 'epoch': 1.0}
{'loss': 0.4867, 'grad_norm': 2.1565563678741455, 'learning_rate': 9.868287740628168e-06, 'epoch': 1.52}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.39983832836151123, 'eval_accuracy': 0.8071942446043165, 'eval_f1': 0.654639175257732, 'eval_precision': 0.7257142857142858, 'eval_recall': 0.596244131455399, 'eval_runtime': 2.8976, 'eval_samples_per_second': 47.971, 'eval_steps_per_second': 6.212, 'epoch': 2.0}


  0%|          | 0/18 [00:00<?, ?it/s]

{'eval_loss': 0.39253881573677063, 'eval_accuracy': 0.8215827338129497, 'eval_f1': 0.6884422110552764, 'eval_precision': 0.7405405405405405, 'eval_recall': 0.6431924882629108, 'eval_runtime': 2.8944, 'eval_samples_per_second': 48.024, 'eval_steps_per_second': 6.219, 'epoch': 3.0}
{'train_runtime': 1332.6178, 'train_samples_per_second': 5.918, 'train_steps_per_second': 0.741, 'train_loss': 0.41679346766756903, 'epoch': 3.0}


TrainOutput(global_step=987, training_loss=0.41679346766756903, metrics={'train_runtime': 1332.6178, 'train_samples_per_second': 5.918, 'train_steps_per_second': 0.741, 'total_flos': 85585447238082.0, 'train_loss': 0.41679346766756903, 'epoch': 3.0})

In [42]:
predictions = trainer.predict(tokenized_test)
probs = sigmoid(torch.from_numpy(predictions.predictions))
print("PROBS: ", probs)
print("LABELS: ", torch.tensor(tokenized_test['label'])) # trues

for thresh in [0.35, 0.4, 0.45, 0.5]:
  # binarize predictions
  binary_predictions = (probs >= thresh).long()
  print("THRESH = ", thresh)
  # print("PREDS: ", binary_predictions)

  print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='weighted'))
  print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='micro'))
  print(f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='macro'))
  print("==============")

  0%|          | 0/18 [00:00<?, ?it/s]

PROBS:  tensor([[0.0378, 0.8583, 0.0325, 0.4586, 0.0154],
        [0.0133, 0.8012, 0.0365, 0.2091, 0.0846],
        [0.0199, 0.8627, 0.0293, 0.3508, 0.0316],
        [0.0315, 0.6390, 0.0496, 0.3358, 0.0620],
        [0.1850, 0.9053, 0.0282, 0.8332, 0.2443],
        [0.0916, 0.9341, 0.0191, 0.2258, 0.8204],
        [0.0808, 0.9634, 0.0133, 0.3555, 0.3780],
        [0.0279, 0.4253, 0.1374, 0.3908, 0.0208],
        [0.0623, 0.9753, 0.0134, 0.4915, 0.4299],
        [0.0227, 0.9051, 0.0305, 0.1291, 0.5461],
        [0.0277, 0.9258, 0.0387, 0.6656, 0.0227],
        [0.0825, 0.9478, 0.0231, 0.3364, 0.8232],
        [0.2659, 0.8695, 0.0582, 0.2120, 0.9352],
        [0.0442, 0.9511, 0.0126, 0.3727, 0.2956],
        [0.0223, 0.5683, 0.1825, 0.5197, 0.0133],
        [0.6112, 0.9126, 0.0261, 0.7806, 0.2205],
        [0.1935, 0.9613, 0.0213, 0.3065, 0.8414],
        [0.0267, 0.9572, 0.0230, 0.5967, 0.0657],
        [0.0349, 0.3873, 0.1351, 0.0844, 0.6790],
        [0.3861, 0.7610, 0.0494, 0.7351, 0

In [ ]:
nums = np.round(np.linspace(.3, .7, 40), 2)
final_nums = np.append(nums, 0.5)

best_thresh = 0
best_f1 = 0

print(final_nums)

for curr_thresh in final_nums:
  # binarize predictions
  binary_predictions = (probs >= curr_thresh).long()
  
  # print("PREDS: ", binary_predictions)

  curr_f1 = f1_score(y_true=tokenized_test['label'], y_pred=binary_predictions, average='weighted')
  if curr_f1 > best_f1:
    best_f1 = curr_f1
    best_thresh = curr_thresh
  # print("THRESH = ", curr_thresh, " F1 = ", curr_f1)
  
print(best_f1)
print(best_thresh)

[0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4  0.41 0.42 0.43
 0.44 0.45 0.46 0.47 0.48 0.49 0.51 0.52 0.53 0.54 0.55 0.56 0.57 0.58
 0.59 0.6  0.61 0.62 0.63 0.64 0.65 0.66 0.67 0.68 0.69 0.7  0.5 ]
0.7115850782304198
0.31
